In [1]:
import numpy as np
import pandas as pd

In [2]:
def donchian(prices, period):
    '''Calculate upper, lower, & middle Donchian lines.'''
    df = pd.DataFrame(prices.copy())
    df['upr'] = df.high.rolling(period).max().shift(periods=1)
    df['lwr'] = df.low.rolling(period).min().shift(periods=1)
    df['mid'] = 0.5 * (df.upr + df.lwr)
    return df


def weekly(exchange, tidm):
    '''Generate weekly prices from SharePad csv file of daily prices.'''
    df = pd.read_csv(
        f'{exchange}_{tidm}_prices.csv',
        header=0,
        names=['date', 'open', 'high', 'low', 'close'],
        index_col=0,
        usecols=[0, 1, 2, 3, 4],
        parse_dates=True,
        dayfirst=True,
    )
    df = df.sort_index()
    functions = dict(open='first', high='max', low='min', close='last')
    df = df.resample('W-FRI').agg(functions)
    df = df / 100
    return df

In [3]:
# Trade parameters.
exchange = 'LSE'
tidm = 'HSV'
periods = [48, 24, 12, 6]  # System look back periods.
position_size = 7500  # Position size in major currency unit.
risk_pct = 0.2  # Percentage risk per trade.
commission = 11.95  # Commission per trade.
sduty = 0.5  # Stamp Duty percentage.

In [4]:
# Import weekly prices.
prices = weekly(exchange, tidm)

In [5]:
# Donchian channels.
dc = []
for period in periods:
    dc.append(donchian(prices, period))

In [6]:
# Trade signals.
for sys, df in enumerate(dc):
    df['buy'] = np.where(df.close > df.upr, 1, 0)
    df['sell'] = np.where(df.close < df.mid, 1, 0)
    df['state'] = 0
    for i in range(period, len(df)):
        if df.loc[df.index[i], 'buy'] == 1 and df.loc[df.index[i - 1], 'state'] == 0:
            df.loc[df.index[i], 'state'] = 1
        elif df.loc[df.index[i], 'sell'] == 1:
            df.loc[df.index[i], 'state'] = 0
        else:
            df.loc[df.index[i], 'state'] = df.loc[df.index[i - 1], 'state']
    df['entry'] = np.where(np.logical_and(df.state == 1, df.state.shift(periods=1) == 0), 1, 0)
    df['exit'] = np.where(np.logical_and(df.state == 0, df.state.shift(periods=1) == 1), 1, 0)
dc[0].loc['2001-09':'2002-09']

,open,high,low,close,upr,lwr,mid,buy,sell,state,entry,exit
date,,,,,,,,,,,,
2001-09-07,1.11140,1.11460,1.10600,1.10820,1.2062,0.77538,0.990790,0,0,0,0,0
2001-09-14,1.10820,1.10820,1.08230,1.08770,1.2062,0.77538,0.990790,0,0,0,0,0
2001-09-21,1.08770,1.08770,0.93692,0.94769,1.2062,0.79692,1.001560,0,1,0,0,0
2001-09-28,0.94231,0.99077,0.93692,0.99077,1.2062,0.80231,1.004255,0,1,0,0,0
2001-10-05,0.99077,1.04460,0.99077,1.04460,1.2062,0.80231,1.004255,0,0,0,0,0
2001-10-12,1.04460,1.10710,1.04460,1.10710,1.2062,0.80231,1.004255,0,0,0,0,0
2001-10-19,1.10710,1.12000,1.10380,1.12000,1.2062,0.83462,1.020410,0,0,0,0,0
2001-10-26,1.12000,1.19540,1.12000,1.19000,1.2062,0.86692,1.036560,0,0,0,0,0
2001-11-02,1.19000,1.20620,1.18460,1.19000,1.2062,0.86692,1.036560,0,0,0,0,0
